# Essential matrices
In the last notebook you computed point correspondences between images using SIFT descriptors and a brute force matching scheme similar to what was used for image stitching.  With these correspondences in hand, we could, in principle, apply the triangulation code developed earlier in order to find the 3D location of all these points.  Or could we?  Triangulation was possible because we already had pre-defined ground control points with which to compute a camera matrix.  However, producing these ground control points is extremely laborious: for each image that we might care to analyze, we must find (manually) at least 3 (and sometimes more) examples for which we know a correspondence between real world coordinates and image coordinates.  This is often not desirable (or even possible).

Instead, we can develop a method for finding the geometric transformation between two cameras using our automatically detected point correspondences.  Note that we will not (without further information), be able to determine the scale of this transformation (e.g. how far the cameras are from each other).  However, we can accurately recover a relative motion, along with the exact rotation matrix.  At the same time, this estimation process will allow us to throw out spurious correspondences, just as we did when using RANSAC to estimate homographies for image stitching.  

To understand how this may be done, we need to understand so-called *epipolar geometry*.  Szeliski Chapter 7 explores this topic in detail: please read it.  While this sounds a little bit intimidating, the underlying principle is relatively simple: 
<img src=epipolar.jpg>
for cameras positioned as such, the ray on which a point lies in the first image is a line in the second image, and vice versa.  These rays, along with a line connecting the optical centers of each camera (the epipole), form a plane in 3D space called an epipolar plane.  Using the constraint that both rays plus the epipole must be coplanar, we can come up with the constraint that 
$$
\mathbf{u}'^T \mathbf{E} \mathbf{u} = 0,
$$
where $\mathbf{u}',\mathbf{u}$ are the image coordinates of point $\mathbf{X}$.  We can then show the useful property that 
$$
\mathbf{E} = [\mathbf{t}]_\times \mathbf{R},
$$
where $[\cdot]_\times$ is the matrix cross product operator, and $\mathbf{t}$ and $\mathbf{R}$ are the relative rotations and translations between two cameras.  These matrices can then be recovered through singular value decomposition from the essential matrix (with $\mathbf{t}$ being defined up to a constant).  

Using the constraint given above, we can form a linear set of homogeneous equations for the entries in $\mathbf{E}$ in which each of these equations depends upon the points in a set of point correspondences.  As in the case of computing homographies, this computation is sensitive to outliers, and thus care must be taken to avoid them.  Of course, we already know how to do this sort of thing using RANSAC.  However, there are a few more subtleties associate with computing the essential matrix $\mathbf{E}$, that must be taken care of.  

Rather than compute this matrix ourselves, we will utilize the functionality provided by OpenCV, which performs the assembly, solution, and outlier detection procedure into a simple function call:

In [ ]:
E,inliers = cv2.findEssentialMat(uv1,uv2,K,method=cv2.RANSAC,threshold=4)

Once the essential matrix is found, we can recover the translation and rotation matrix up to a scale using:

In [ ]:
n_in,R,t,_ = cv2.recoverPose(E,uv1,uv2,mask=inliers)

You'll note that this function only returns a single rotation and translation: this method assumes that the first camera has canonical pose $\mathbf{t} = \mathbf{0}$ and $\mathbf{R} = \mathbf{I}$.  Alteratively, we can immediately define the camera matrices

In [ ]:
P_0 = K @ np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0]])
P_1 = K @ np.hstack((R,t))

where $P_0$ and $P_1$ are the camera matrices, and $K$ is the matrix of camera intrinsics
$$
K = \begin{bmatrix} f & 0 & c_u \\
                    0 & f & c_v \\
                    0 & 0 & 1 \end{bmatrix}
$$
